# Grove LED Bar Example

This examples shows how to use the Grove LED bar on the Pynq-Z1 board.

A [Grove LED Bar](http://www.seeedstudio.com/depot/Grove-LED-Bar-v20-p-2474.html) and Pynq Grove Adapter adapter, or Pynq Shield is required. In this example, the Pynq Grove Adapter adapter is used, but the notebook can be modified to use the Pynq Shield.

The LED bar is attached to the J1 connection on the Pynq Grove Adapter, which is connnected to PMODA on the Pynq-Z1 board. 

The LED bar has 10 LEDs. 8 Green LEDs, 1 Orange LED, and 1 Red LED. The brightness for each LED can be set independantly. 

In [ ]:
# Make sure the base overlay is loaded
from pynq import Overlay
Overlay("base.bit").download()

### 1. Instantiate and reset LED Bar
The IOP (Pmod IOP (A or B) or Arduino IOP) must be selected, and the pins on the Pmod connector, or Pynq shiled must be specified. 

In [ ]:
from pynq.iop import Grove_LEDbar

# Instantiate Grove LED Bar on PMODA and on Pmod2Grove G1
# PMODA = 1, PMODB = 2, Arduino = 3
# Pynq Grove Adapter; J1 = [0,4], J2 = [1,5], J3 = [7,3], J4 = [6,2]
# The LED bar can be connected to G1-7 on the Pynq Shield:
# G1 = [2,3], G2 = [3,4], G3 = [4,5], G4 = [6,7], G5 = [8,9], G6 = [10,11], G7 = [12,13]
ledbar = Grove_LEDbar(1,[0,4])
ledbar.reset()

### 2. Turn individual LEDs on or off

Write a 10-bit binary pattern, with each bit representing the corresponding LED. 1 = on, 0 = off

In [ ]:
#from pynq.pmods import pmod_const
from time import sleep

# Light up different bars in a loop
for i in range(2):
    ledbar.write_binary(0b1010100000)
    sleep(0.5)
    ledbar.write_binary(0b0000100100)
    sleep(0.5)
    ledbar.write_binary(0b1010101110)
    sleep(0.5)
    ledbar.write_binary(0b1111111110)
    sleep(0.5)

### 3. Set LEDs individually with different brightness levels

The brightness of each LED can be set individually by writing a list of 10x 8-bit values to the LED bar. 0 is off, 0xff is full brightness. 

In [ ]:
# Brightness 0-255
HIGH = 0xFF
MED  = 0xAA
LOW  = 0x01
OFF  = 0X00

brightness = [OFF, OFF, OFF, LOW, LOW, MED, MED, HIGH, HIGH, HIGH]

ledbar.write_brightness(0b1111111111,brightness)

### 4.  Set the "level" or the number of LEDs which are set 

A number or level of LEDs can be turned on, started from either end of the LED bar. For example, this could be used to indicate the level of something being measured.

write_level(level, bright_level, green_to_red)

* level is the number of LEDs that are on.
* bright_level [0-10] is the level of brightness
* green_to_red = 1 means the LEDs start being lit from the "green" end of the LED bar
* green_to_red = 0 means the LEDs start being lit from the "red" end of the LED bar.

e.g. ledbar.write_level(5,4,1) will light 5 LEDs, to brightness 4 (out of 10) and will start from the Green LED (the LED furthest away from Grove connector on the LED bar module.)


In [ ]:
for i in range (1,11):
    ledbar.write_level(i,3,0)
    sleep(0.3)
for i in range (1,10):
    ledbar.write_level(i,3,1)
    sleep(0.3)    

### 5. Simple example of controlling the LED Bar from the board buttons

The "level" of the LEDs can be set using buttons on the board. 

* Button 0 to increase level
* Button 1 to decrease level
* Button 3 to exit

In [ ]:
from pynq.board import Button

btns = [Button(index) for index in range(4)] 
i = 1
ledbar.reset()

done = False
while not done:
    if (btns[0].read()==1):
        sleep(0.2)
        ledbar.write_level(i,2,1)
        i = min(i+1,9)
    elif (btns[1].read()==1):
        sleep(0.2)
        i = max(i-1,0)
        ledbar.write_level(i,2,1)
    elif (btns[3].read()==1):
        ledbar.reset()
        done = True